In [48]:
from F4 import PNL_per_day, DumpCSV_and_MesToTele, position_input, position_report, BacktestInformation
import pandas as pd
import datetime
import numpy as np
import time
import requests
from time import sleep
from ta.momentum import RSIIndicator, AwesomeOscillatorIndicator
from ta.trend import MACD, CCIIndicator, EMAIndicator
from ta.volatility import BollingerBands
from ta.volume import ChaikinMoneyFlowIndicator
import warnings
warnings.filterwarnings('ignore')
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True, connected=True)

In [49]:
data = pd.read_csv('.\data1mins.csv')

In [50]:
data

Loading ITables v2.5.2 from the internet... (need help?)


In [51]:
data.Date = pd.to_datetime(data.Date, errors = 'coerce')
data = data.set_index("Date")
data = data.resample('10Min').agg({'Open': 'first', 'Close': 'last', 'High': 'max', 'Low': 'min', 'Volume': 'sum'}).dropna()
data = data.reset_index()

In [52]:
from scipy import signal
from scipy.signal import butter,lfilter 

def lowpass_filter_period_only(x, cutoff_period: int = 20, order: int = 2, zero_phase: bool = False):
    cutoff_period = max(int(cutoff_period), 3)  # kẹp tối thiểu 3
    Wn = 2.0 / float(cutoff_period)
    Wn = min(max(Wn, 1e-6), 0.999999)          # đảm bảo (0,1)
    sos = butter(order, Wn, btype="low", output="sos")
    if zero_phase:
        y = signal.sosfiltfilt(sos, pd.Series(x).values, method="gust")
    else:
        y = signal.sosfilt(sos, pd.Series(x).values)
    return pd.Series(y, index=pd.Series(x).index)


In [53]:
df = pd.read_csv('.\expiration_date.csv')
df.Date = pd.to_datetime(df['Date'])
expiration_date = set(map(lambda x: x.date(), df['Date']))
expiration_date

{datetime.date(2034, 8, 17),
 datetime.date(2067, 8, 18),
 datetime.date(2245, 3, 20),
 datetime.date(2060, 6, 17),
 datetime.date(2239, 7, 18),
 datetime.date(2164, 7, 19),
 datetime.date(2152, 6, 15),
 datetime.date(2078, 5, 19),
 datetime.date(2095, 5, 19),
 datetime.date(2108, 8, 16),
 datetime.date(2028, 8, 17),
 datetime.date(2050, 9, 15),
 datetime.date(2121, 7, 17),
 datetime.date(2187, 3, 15),
 datetime.date(2219, 12, 16),
 datetime.date(2055, 2, 18),
 datetime.date(2070, 9, 18),
 datetime.date(2134, 6, 17),
 datetime.date(2198, 7, 19),
 datetime.date(2155, 9, 18),
 datetime.date(2228, 2, 21),
 datetime.date(2192, 5, 17),
 datetime.date(2070, 11, 20),
 datetime.date(2143, 10, 17),
 datetime.date(2250, 4, 18),
 datetime.date(2043, 6, 18),
 datetime.date(2079, 2, 16),
 datetime.date(2147, 10, 19),
 datetime.date(2246, 3, 19),
 datetime.date(2189, 10, 15),
 datetime.date(2029, 4, 19),
 datetime.date(2017, 6, 15),
 datetime.date(2249, 6, 21),
 datetime.date(2182, 5, 16),
 datetime

In [ ]:
# AverageTrueRange, CCIIndicator, IchimokuIndicator, MACD, RSI, ADXIndicator, RSI, stochastic,
def alpha(data, params):
    data['Close_filtered'] = lowpass_filter_period_only(data['Close'], params['t1'])

    #tính chỉ báo 
    data['RSI'] = RSIIndicator(close=data['Close'], window = params['w1']).rsi()
    data['Boll'] = BollingerBands
    
    
    
    pos = 0
    position = []
    for i in range(len(data)):
        #
        
        RSI = data['RSI'].values[i]
        
        

        #
        if pos == 0:
            #điều kiện enter short
            if RSI < params['t2'] :#13. threshold to enter short 30
                pos = -1
            #điều kiện vào long
            if RSI > params['t3']:#81. threshold to enter long 70
                pos = 1
        elif pos == -1:
            #thoát short
            if RSI > params['t4']:#59. threshold to exit short
                pos = 0
        else:
            #thoát long
            if RSI < params['t5']:#37. threshold to exit long
                pos = 0
        if data['Date'].iloc[i].time() == datetime.time(14, 25):
            #thoát short cuối ngày(CÓ THỂ HOẶC KHÔNG)
            if pos == - 1:
                pos = 0
        if data['Date'].iloc[i].time() == datetime.time(14, 45):
            if len(position) > 0:
                #reset vị thế ngày đáo hạn
                if data['Date'].iloc[i].date() in expiration_date:
                    pos = 0

        position.append(pos)
    #mục đích cuối cùng vẫn là cột pos(position)
    data['pos'] = position
    return data


In [63]:
params = {'t1': 0.75, 't2': 33, 't3': 71, 't4': 43, 't5': 51, 'w1': 10}

In [64]:
ds = alpha(data, params)
ds

Loading ITables v2.5.2 from the internet... (need help?)


In [65]:
backtestInfo = BacktestInformation(ds['Date'], ds['pos'], ds['Close'], fee = 0.3) 
pnl = backtestInfo.Plot_PNL()

IndexError: single positional indexer is out-of-bounds

In [ ]:
ds.pos

Loading ITables v2.5.2 from the internet... (need help?)


In [ ]:
pnl

Loading ITables v2.5.2 from the internet... (need help?)


In [ ]:
merged = pd.merge(ds, pnl, left_on="Date",right_on= "Datetime", how="inner")
merged

Loading ITables v2.5.2 from the internet... (need help?)


In [ ]:
df = pd.DataFrame(ds)

# select Date and pos
df_result = df[["Date", "Close", "pos"]]
df_result['pos_fin'] = df_result['pos']
df_result = df_result.drop('pos', axis = 1)
df_result

Loading ITables v2.5.2 from the internet... (need help?)


In [ ]:
df_result.to_csv("alpha4_pos.csv", index=False)